In [1]:
import cleverhans
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from keras.utils import np_utils
from sklearn.metrics import confusion_matrix

In [2]:
# new Data preparation

# Training Dataset
data = pd.read_csv("./dataset/UNSW_2018_IoT_Botnet_Final_10_best_Training.csv")

x_features = data[['seq','stddev','N_IN_Conn_P_SrcIP', 'min', 'state_number', 'mean', 'N_IN_Conn_P_DstIP',
       'drate', 'srate', 'max', 'category', 'subcategory']]

# Label Encoding the target columns
le = LabelEncoder()
x_features['category'] = le.fit_transform(x_features['category'])
x_features['subcategory'] = le.fit_transform(x_features['subcategory'])

x = x_features.values
x = preprocessing.scale(x)
y = data['attack'].values

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

y_train_cat = np_utils.to_categorical(y_train)
y_test_cat = np_utils.to_categorical(y_test)

C:\Users\sumsu\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
C:\Users\sumsu\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


In [3]:
model = tf.keras.Sequential([
        tf.keras.layers.Dense(200, input_dim=x.shape[1], activation=tf.nn.relu),
        tf.keras.layers.Dense(500, activation=tf.nn.relu),
        tf.keras.layers.Dense(200, activation=tf.nn.relu),
        tf.keras.layers.Dense(len(np.unique(y))),
        # We seperate the activation layer to be able to access
        # the logits of the previous layer later
        tf.keras.layers.Activation(tf.nn.softmax)
        ])

model.compile(optimizer='adam',
              loss= 'categorical_crossentropy',
              metrics=['accuracy'])

In [4]:
y_train_cat = np_utils.to_categorical(y_train)
y_test_cat = np_utils.to_categorical(y_test)

model.fit(X_train, y_train_cat, epochs=10, validation_split=0.2)

Epoch 1/10
58697/58697 [==============================] - 264s 4ms/step - loss: 0.0024 - accuracy: 0.9999 - val_loss: 7.7015e-04 - val_accuracy: 0.9999
Epoch 2/10
58697/58697 [==============================] - 240s 4ms/step - loss: 0.0010 - accuracy: 1.0000 - val_loss: 2.5392e-04 - val_accuracy: 1.0000
Epoch 3/10
58697/58697 [==============================] - 240s 4ms/step - loss: 4.8794e-04 - accuracy: 1.0000 - val_loss: 3.5963e-05 - val_accuracy: 1.0000
Epoch 4/10
58697/58697 [==============================] - 241s 4ms/step - loss: 7.5967e-04 - accuracy: 1.0000 - val_loss: 0.0010 - val_accuracy: 1.0000
Epoch 5/10
58697/58697 [==============================] - 242s 4ms/step - loss: 4.9258e-04 - accuracy: 1.0000 - val_loss: 5.3667e-10 - val_accuracy: 1.0000
Epoch 6/10
58697/58697 [==============================] - 243s 4ms/step - loss: 0.0016 - accuracy: 1.0000 - val_loss: 1.0941e-05 - val_accuracy: 1.0000
Epoch 7/10
58697/58697 [==============================] - 237s 4ms/step - loss: 

In [5]:
test_loss, test_acc = model.evaluate(X_test, y_test_cat)

print('Test accuracy:', test_acc)

18343/18343 [==============================] - 32s 2ms/step - loss: 7.0514e-04 - accuracy: 1.0000
Test accuracy: 0.9999914765357971


In [6]:
y_pred = model.predict(X_test)
y_pred=np.argmax(y_pred, axis=1)
cm_org = confusion_matrix(y_test, y_pred)
print("*"*50)
print("Original confusion matrix")
print(cm_org)

18343/18343 [==============================] - 24s 1ms/step
**************************************************
Original confusion matrix
[[    62      5]
 [     0 586897]]


In [7]:
# Import the attack
from cleverhans.future.tf2.attacks import fast_gradient_method

#The attack requires the model to ouput the logits
logits_model = tf.keras.Model(model.input, model.layers[-1].output)

epsilon = 0.1
adv_fgsm_x = fast_gradient_method(logits_model,
                                  X_test,
                                  epsilon,
                                  np.inf,
                                  targeted=False)

In [8]:
y_pred = model.predict(adv_fgsm_x)
y_pred=np.argmax(y_pred, axis=1)
cm_adv = confusion_matrix(y_test, y_pred)
print("*"*20)
print("Attacked confusion matrix")
print(cm_adv)

18343/18343 [==============================] - 15s 788us/step
********************
Attacked confusion matrix
[[    53     14]
 [     0 586897]]


In [9]:
print(adv_fgsm_x)

tf.Tensor(
[[-1.2842484  -1.10368447 -0.7195503  ... -1.6224652   0.75246724
  -0.86685956]
 [ 0.70762393 -0.9182629  -2.27755391 ... -1.45260141 -0.83068184
  -0.86685956]
 [-1.51456499 -1.10368447 -1.62155239 ... -1.6224652   3.91876542
  -2.57865064]
 ...
 [-0.57398133  1.2822774   0.71545303 ...  0.5904311  -0.83068184
  -0.86685956]
 [ 1.74030783 -0.95340555  0.71545303 ...  0.4423308   0.75246724
   0.84493152]
 [-1.33167079 -1.09202003 -2.48255439 ... -1.61239112 -0.83068184
  -0.86685956]], shape=(586964, 12), dtype=float64)
